In [220]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import gridspec            # to arrange the plots nicely
import pandas as pd

%matplotlib inline

In [221]:
#=====================================
# Load txt file to pandas dataframe
#=====================================
data_raw = pd.read_csv('mu-g4.txt', sep='\t', names=['tdc11', 'tdc12', 'tdc13', 'tdc14', 'tdc15', 'tdc16', 'tdc17', 'tdc18',
                                                        'tdc21', 'tdc22', 'tdc23', 'tdc24', 'tdc25', 'tdc26', 'tdc27', 'tdc28',
                                                       ])
data_raw = data_raw.rename(columns={'tdc11' : 'p0_1',
                            'tdc12' : 'p1_1',
                            'tdc13' : 'p2_1',
                            'tdc14' : 'start2',
                            'tdc24' : 'p0_2',
                            'tdc25' : 'p1_2',
                            'tdc26' : 'p2_2',
                            'tdc23' : 'stop2',
                           })
print('Total number of events:', len(data_raw))
data_raw.head()

Total number of events: 1014917


,p0_1,p1_1,p2_1,start2,tdc15,tdc16,tdc17,tdc18,tdc21,tdc22,stop2,p0_2,p1_2,p2_2,tdc27,tdc28
0,4095,4095,4095,3477,4095,4095,4095,4095,3839,4095,1679,4095,4095,4095,4095,4095
1,4095,4095,4095,3477,4095,4095,4095,4095,3839,4095,1680,4095,4095,4095,4095,4095
2,4095,4095,4095,3479,4095,4095,4095,4095,4095,4095,1681,4095,4095,4095,4095,4095
3,4095,4095,4095,3478,4095,4095,4095,4095,4095,4095,1680,4095,4095,4095,4095,4095
4,4095,4095,4095,3478,4095,4095,4095,4095,4095,4095,1680,4095,4095,4095,4095,4095


In [222]:
# keeps only necessary

data = data_raw[['p0_1','p1_1','p2_1','start2','p0_2','p1_2','p2_2','stop2']]
data.head()

,p0_1,p1_1,p2_1,start2,p0_2,p1_2,p2_2,stop2
0,4095,4095,4095,3477,4095,4095,4095,1679
1,4095,4095,4095,3477,4095,4095,4095,1680
2,4095,4095,4095,3479,4095,4095,4095,1681
3,4095,4095,4095,3478,4095,4095,4095,1680
4,4095,4095,4095,3478,4095,4095,4095,1680


In [223]:
# loads fitted parameters from parameters.txt

par=pd.read_csv('parameters.txt', sep=' ', names=['tdc','a','b'])
df_par=par.set_index('tdc')
df_par.head()

,a,b
tdc,,
tdc11,0.759454,0.852281
tdc12,0.771293,-3.229693
tdc13,0.751868,1.141473
tdc14,0.763373,1.004555
tdc15,0.743359,3.259955


In [206]:
df_par.loc['tdc14', 'b']

1.00455541

In [224]:
#=====================================
# Convert tdc units into ns
#=====================================
def tdc_to_ns(channel, tdc_unit):
    ns = (tdc_unit - df_par.loc[channel, 'b']) / df_par.loc[channel, 'a']
    return ns

#=====================================
# Only one valide time for each detector
#=====================================
def merge_time(d,number):
    if d['p%i_1ns'%(number)]==4095:
        return df['p%i_2_ns'%(number)]
    else:
        return df['p%i_1_ns'%(number)]
    
#=====================================
# Add actual time variables to the dataframe
#=====================================
def conv_time(d):
    d['start2_ns'] = tdc_to_ns('tdc14', d.start2)
    d['p0_1_ns'] = tdc_to_ns('tdc11', d.p0_1)
    d['p1_1_ns'] = tdc_to_ns('tdc12', d.p1_1)
    d['p2_1_ns'] = tdc_to_ns('tdc13', d.p2_1)
    d['p0_2_ns'] = tdc_to_ns('tdc24', d.p0_2) + d.start2
    d['p1_2_ns'] = tdc_to_ns('tdc25', d.p1_2) + d.start2
    d['p2_2_ns'] = tdc_to_ns('tdc26', d.p2_2) + d.start2
    
    #df['p0_time'] = df.apply(lambda df: merge_time_p0(df),axis=1)
    #df['p1_time'] = df.apply(lambda df: merge_time_p1(df),axis=1)
    #df['p2_time'] = df.apply(lambda df: merge_time_p2(df),axis=1)
    return d

In [14]:
type(df_par.loc['tdc11','a'])

numpy.float64

In [225]:
conv_time(data)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

,p0_1,p1_1,p2_1,start2,p0_2,p1_2,p2_2,stop2,start2_ns,p0_1_ns,p1_1_ns,p2_1_ns,p0_2_ns,p1_2_ns,p2_2_ns
0,4095,4095,4095,3477,4095,4095,4095,1679,4553.469917,5390.911332,5313.451213,5444.918664,8714.222059,8654.209029,8512.094175
1,4095,4095,4095,3477,4095,4095,4095,1680,4553.469917,5390.911332,5313.451213,5444.918664,8714.222059,8654.209029,8512.094175
2,4095,4095,4095,3479,4095,4095,4095,1681,4556.089869,5390.911332,5313.451213,5444.918664,8716.222059,8656.209029,8514.094175
3,4095,4095,4095,3478,4095,4095,4095,1680,4554.779893,5390.911332,5313.451213,5444.918664,8715.222059,8655.209029,8513.094175
4,4095,4095,4095,3478,4095,4095,4095,1680,4554.779893,5390.911332,5313.451213,5444.918664,8715.222059,8655.209029,8513.094175
5,4095,4095,4095,3477,4095,4095,4095,1679,4553.469917,5390.911332,5313.451213,5444.918664,8714.222059,8654.209029,8512.094175
6,4095,4095,4095,3478,4095,4095,4095,1679,4554.779893,5390.911332,5313.451213,5444.918664,8715.222059,8655.209029,8513.094175
7,4095,4095,4095,3478,4095,4095,4095,1680,4554.779893,5390.911332,5313.451213,5444.918664,8715.222059,8655.209029,8513.094175
8,4095,4095,4095,3478,4095,4095,4095,1679,4554.779893,5390.911332,5313.451213,5444.918664,8715.222059,8655.209029,8513.094175
9,4095,4095,4095,3478,4095,4095,4095,1680,4554.779893,5390.911332,5313.451213,5444.918664,8715.222059,8655.209029,8513.094175


In [189]:
data

,p0_1,p1_1,p2_1,start2,p0_2,p1_2,p2_2,stop2,p0_1_ns,p1_1_ns,p2_1_ns,p0_2_ns,p1_2_ns,p2_2_ns
0,4095,4095,4095,2027.951967,4095,4095,4095,1679,3110.815305,3155.216658,3080.039746,5237.479154,5268.969491,5357.210779
1,4095,4095,4095,2027.951967,4095,4095,4095,1680,3110.815305,3155.216658,3080.039746,5237.479154,5268.969491,5357.210779
2,4095,4095,4095,2029.117443,4095,4095,4095,1681,3110.815305,3155.216658,3080.039746,5238.644630,5270.134967,5358.376255
3,4095,4095,4095,2028.534705,4095,4095,4095,1680,3110.815305,3155.216658,3080.039746,5238.061892,5269.552229,5357.793517
4,4095,4095,4095,2028.534705,4095,4095,4095,1680,3110.815305,3155.216658,3080.039746,5238.061892,5269.552229,5357.793517
5,4095,4095,4095,2027.951967,4095,4095,4095,1679,3110.815305,3155.216658,3080.039746,5237.479154,5268.969491,5357.210779
6,4095,4095,4095,2028.534705,4095,4095,4095,1679,3110.815305,3155.216658,3080.039746,5238.061892,5269.552229,5357.793517
7,4095,4095,4095,2028.534705,4095,4095,4095,1680,3110.815305,3155.216658,3080.039746,5238.061892,5269.552229,5357.793517
8,4095,4095,4095,2028.534705,4095,4095,4095,1679,3110.815305,3155.216658,3080.039746,5238.061892,5269.552229,5357.793517
9,4095,4095,4095,2028.534705,4095,4095,4095,1680,3110.815305,3155.216658,3080.039746,5238.061892,5269.552229,5357.793517
